In [3]:
#Author : Santosh Kumar Udayakumar
#Function : Extract the SL data stop files and mine the necessary information . 
from lxml import etree

# Load the NeTEx file
with open('sl_data/_stops.xml', 'rb') as file:
    tree = etree.parse(file)

# Define namespaces

namespaces = {
    'netex': 'http://www.netex.org.uk/netex',
    'gml': 'http://www.opengis.net/gml/3.2',
    'siri': 'http://www.siri.org.uk/siri',
    'sam': 'http://www.samtrafiken.se/netex'
}

group_of_stop_places = tree.xpath('//netex:GroupOfStopPlaces', namespaces=namespaces)

stops = []

for group in group_of_stop_places:
    name_elem = group.find('netex:Name', namespaces=namespaces)
    description_elem = group.find('netex:Description', namespaces=namespaces)
    private_code_elem = group.find('netex:PrivateCode', namespaces=namespaces)
    latitude_elem = group.find('netex:Centroid/netex:Location/netex:Latitude', namespaces=namespaces)
    longitude_elem = group.find('netex:Centroid/netex:Location/netex:Longitude', namespaces=namespaces)

    group_info = {
        'Group ID': group.get('id'),
        'Name': name_elem.text if name_elem is not None else '',
        'Description': description_elem.text if description_elem is not None else '',
        'Private Code': private_code_elem.text if private_code_elem is not None else '',
        'Latitude': latitude_elem.text if latitude_elem is not None else '',
        'Longitude': longitude_elem.text if longitude_elem is not None else '',
    }
    
    stops.append(group_info)

# Create a DataFrame from the extracted data
stops_df = pd.DataFrame(stops)


In [12]:
stop_points

[]

In [24]:
import pandas as pd
import xml.etree.ElementTree as ET

# Parse the XML file
xml_file = 'sl_data/line_1_814_9011001081400000.xml'  # Replace with the path to your XML file
tree = ET.parse(xml_file)
root = tree.getroot()

# Define the namespace to simplify searching elements
ns = {'netex': 'http://www.netex.org.uk/netex'}

# Extract routes data
routes_data = []
for route in root.findall('.//netex:Route', ns):
    route_id = route.attrib.get('id')
    route_name = route.find('netex:Name', ns).text
    direction = route.find('netex:DirectionType', ns).text
    
    # Extract points in sequence for each route
    points_in_sequence = []
    for point in route.findall('.//netex:PointOnRoute', ns):
        point_id = point.attrib.get('id')
        order = point.attrib.get('order')
        ref = point.find('netex:RoutePointRef', ns).attrib.get('ref')
        points_in_sequence.append({
            'Point_ID': point_id,
            'Order': order,
            'RoutePointRef': ref
        })
    
    routes_data.append({
        'Route_ID': route_id,
        'Route_Name': route_name,
        'Direction': direction,
        'Points_in_Sequence': points_in_sequence
    })

# Create a DataFrame to store routes data
routes_df = pd.DataFrame(routes_data)

# Extract timetable and service journey data
timetable_data = []
for journey in root.findall('.//netex:ServiceJourney', ns):
    journey_id = journey.attrib.get('id')
    transport_mode = journey.find('netex:TransportMode', ns).text
    
    # Extract operator and day types
    operator_ref = journey.find('.//netex:OperatorRef', ns).attrib.get('ref')
    day_type_ref = journey.find('.//netex:DayTypeRef', ns).attrib.get('ref')
    
    # Extract passing times for each stop in the journey
    passing_times = []
    for passing_time in journey.findall('.//netex:TimetabledPassingTime', ns):
        stop_ref = passing_time.find('netex:StopPointInJourneyPatternRef', ns).attrib.get('ref')
        arrival_time = passing_time.find('netex:ArrivalTime', ns)
        departure_time = passing_time.find('netex:DepartureTime', ns)
        passing_times.append({
            'StopPointInJourneyPatternRef': stop_ref,
            'ArrivalTime': arrival_time.text if arrival_time is not None else None,
            'DepartureTime': departure_time.text if departure_time is not None else None
        })
    
    timetable_data.append({
        'Journey_ID': journey_id,
        'TransportMode': transport_mode,
        'OperatorRef': operator_ref,
        'DayTypeRef': day_type_ref,
        'PassingTimes': passing_times
    })

# Create a DataFrame to store service journey timetable data
timetable_df = pd.DataFrame(timetable_data)

# Display the DataFrame
print(routes_df.head())   # Routes DataFrame
print(timetable_df.head()) # Timetable and service journey DataFrame


                         Route_ID         Route_Name Direction  \
0  SE:001:Route:14010000577667337  Södra Hedvigslund  outbound   
1  SE:001:Route:14010000577667012       Gullmarsplan   inbound   

                                  Points_in_Sequence  
0  [{'Point_ID': 'SE:001:PointOnRoute:14010000577...  
1  [{'Point_ID': 'SE:001:PointOnRoute:14010000577...  
                                Journey_ID TransportMode  \
0  SE:001:ServiceJourney:14010000597098249           bus   
1  SE:001:ServiceJourney:14010000597100465           bus   
2  SE:001:ServiceJourney:14010000597095530           bus   
3  SE:001:ServiceJourney:14010000597097043           bus   
4  SE:001:ServiceJourney:14010000597099317           bus   

                        OperatorRef  \
0  SE:001:Operator:9013001003000000   
1  SE:001:Operator:9013001003000000   
2  SE:001:Operator:9013001003000000   
3  SE:001:Operator:9013001003000000   
4  SE:001:Operator:9013001003000000   

                                        

In [36]:
import pandas as pd
import xml.etree.ElementTree as ET

# Parse the XML file
xml_file = 'sl_data/line_1_814_9011001081400000.xml'  # Replace with the path to your XML file
tree = ET.parse(xml_file)
root = tree.getroot()

# Define the namespace to simplify searching elements
ns = {'netex': 'http://www.netex.org.uk/netex'}

# Extract routes data
routes_data = []
for route in root.findall('.//netex:Route', ns):
    route_id = route.attrib.get('id')
    route_name = route.find('netex:Name', ns).text
    direction = route.find('netex:DirectionType', ns).text
    
    # Extract points in sequence for each route
    points_in_sequence = []
    for point in route.findall('.//netex:PointOnRoute', ns):
        point_id = point.attrib.get('id')
        order = point.attrib.get('order')
        ref = point.find('netex:RoutePointRef', ns).attrib.get('ref')
        points_in_sequence.append({
            'Point_ID': point_id,
            'Order': order,
            'RoutePointRef': ref
        })
    
    routes_data.append({
        'Route_ID': route_id,
        'Route_Name': route_name,
        'Direction': direction,
        'Points_in_Sequence': points_in_sequence
    })

# Create a DataFrame to store routes data
routes_df = pd.DataFrame(routes_data)

# Extract timetable and service journey data along with public and private codes
timetable_data = []
for journey in root.findall('.//netex:ServiceJourney', ns):
    journey_id = journey.attrib.get('id')
    transport_mode = journey.find('netex:TransportMode', ns).text
    
    # Extract operator, day types, and public/private codes
    operator_ref = journey.find('.//netex:OperatorRef', ns).attrib.get('ref')
    day_type_ref = journey.find('.//netex:DayTypeRef', ns).attrib.get('ref')
    
    # Extract LineRef to find public and private codes
    line_ref = journey.find('.//netex:JourneyPatternRef', ns).attrib.get('ref')
    line = root.find(f".//netex:Line[@id='{line_ref}']", ns)
    
    public_code = line.find('netex:PublicCode', ns).text if line is not None else None
    private_code = line.find('netex:PrivateCode', ns).text if line is not None else None
    
    # Extract passing times for each stop in the journey
    for passing_time in journey.findall('.//netex:TimetabledPassingTime', ns):
        stop_ref = passing_time.find('netex:StopPointInJourneyPatternRef', ns).attrib.get('ref')
        arrival_time = passing_time.find('netex:ArrivalTime', ns)
        departure_time = passing_time.find('netex:DepartureTime', ns)
        
        timetable_data.append({
            'Journey_ID': journey_id,
            'TransportMode': transport_mode,
            'OperatorRef': operator_ref,
            'DayTypeRef': day_type_ref,
            'PublicCode': public_code,
            'PrivateCode': private_code,
            'StopPointInJourneyPatternRef': stop_ref,
            'ArrivalTime': arrival_time.text if arrival_time is not None else None,
            'DepartureTime': departure_time.text if departure_time is not None else None
        })

# Create a DataFrame to store service journey timetable data
timetable_df = pd.DataFrame(timetable_data)

# Display the DataFrames
print(routes_df.head())   # Routes DataFrame
print(timetable_df.head()) # Timetable and service journey DataFrame with codes


                         Route_ID         Route_Name Direction  \
0  SE:001:Route:14010000577667337  Södra Hedvigslund  outbound   
1  SE:001:Route:14010000577667012       Gullmarsplan   inbound   

                                  Points_in_Sequence  
0  [{'Point_ID': 'SE:001:PointOnRoute:14010000577...  
1  [{'Point_ID': 'SE:001:PointOnRoute:14010000577...  
                                Journey_ID TransportMode  \
0  SE:001:ServiceJourney:14010000597098249           bus   
1  SE:001:ServiceJourney:14010000597098249           bus   
2  SE:001:ServiceJourney:14010000597098249           bus   
3  SE:001:ServiceJourney:14010000597098249           bus   
4  SE:001:ServiceJourney:14010000597098249           bus   

                        OperatorRef  \
0  SE:001:Operator:9013001003000000   
1  SE:001:Operator:9013001003000000   
2  SE:001:Operator:9013001003000000   
3  SE:001:Operator:9013001003000000   
4  SE:001:Operator:9013001003000000   

                                        

In [7]:
from lxml import etree
import pandas as pd

# Load the NeTEx file
with open('sl_data/_stops.xml', 'rb') as file:
    tree = etree.parse(file)

# Define namespaces (if applicable)
namespaces = {
    'netex': 'http://www.netex.org.uk/netex',
    'gml': 'http://www.opengis.net/gml/3.2',
    'siri': 'http://www.siri.org.uk/siri',
    'sam': 'http://www.samtrafiken.se/netex'
}

# Extract all StopPlace elements
stop_places = tree.xpath('//netex:StopPlace', namespaces=namespaces)

# Initialize a list to store stop and quay data
stops = []

for stop_place in stop_places:
    # Extract StopPlace details
    stop_place_id = stop_place.get('id')
    name_elem = stop_place.find('netex:Name', namespaces=namespaces)
    private_code_elem = stop_place.find('netex:PrivateCode', namespaces=namespaces)
    latitude_elem = stop_place.find('netex:Centroid/netex:Location/netex:Latitude', namespaces=namespaces)
    longitude_elem = stop_place.find('netex:Centroid/netex:Location/netex:Longitude', namespaces=namespaces)

    # Create a dictionary for StopPlace info
    stop_info = {
        'StopPlace ID': stop_place_id,
        'StopPlace Name': name_elem.text if name_elem is not None else '',
        'StopPlace Private Code': private_code_elem.text if private_code_elem is not None else '',
        'StopPlace Latitude': latitude_elem.text if latitude_elem is not None else '',
        'StopPlace Longitude': longitude_elem.text if longitude_elem is not None else ''
    }

    # Now check for any Quays within this StopPlace
    quays = stop_place.findall('.//netex:Quay', namespaces=namespaces)
    if quays:
        for quay in quays:
            # Extract Quay details
            quay_id = quay.get('id')
            quay_name_elem = quay.find('netex:Name', namespaces=namespaces)
            quay_private_code_elem = quay.find('netex:PrivateCode', namespaces=namespaces)
            quay_latitude_elem = quay.find('netex:Centroid/netex:Location/netex:Latitude', namespaces=namespaces)
            quay_longitude_elem = quay.find('netex:Centroid/netex:Location/netex:Longitude', namespaces=namespaces)

            # Add Quay details to StopPlace info
            quay_info = {
                'Quay ID': quay_id if quay_id else '',
                'Quay Name': quay_name_elem.text if quay_name_elem is not None else '',
                'Quay Private Code': quay_private_code_elem.text if quay_private_code_elem is not None else '',
                'Quay Latitude': quay_latitude_elem.text if quay_latitude_elem is not None else '',
                'Quay Longitude': quay_longitude_elem.text if quay_longitude_elem is not None else ''
            }

            # Merge StopPlace and Quay info into a single row
            combined_info = {**stop_info, **quay_info}
            stops.append(combined_info)

    else:
        # If no quays exist, just add StopPlace info without quay details
        stops.append(stop_info)

# Create a DataFrame from the extracted data
stops_df = pd.DataFrame(stops)

# Display the DataFrame
print(stops_df.head())


                        StopPlace ID   StopPlace Name StopPlace Private Code  \
0  SE:001:StopPlace:9021001070004000      Alhagsvägen                  70004   
1  SE:001:StopPlace:9021001070004000      Alhagsvägen                  70004   
2  SE:001:StopPlace:9021001070006000          Alby IP                  70006   
3  SE:001:StopPlace:9021001070006000          Alby IP                  70006   
4  SE:001:StopPlace:9021001070012000  Hantverkarvägen                  70012   

  StopPlace Latitude StopPlace Longitude                       Quay ID  \
0          59.234952           17.851566  SE:001:Quay:9022001070004001   
1          59.234952           17.851566  SE:001:Quay:9022001070004002   
2          59.237646           17.851894  SE:001:Quay:9022001070006001   
3          59.237646           17.851894  SE:001:Quay:9022001070006002   
4          59.235563           17.836068                           NaN   

     Quay Name Quay Private Code Quay Latitude Quay Longitude  
0  Alhagsv